# CSC 522 Project 
_by Team 21: Raphael Phillips (rjphill4), Sogolsadat Mansouri (smansou2), Rithik Jain (rjain25), and Neeloy Gomes (ntgomes)_

This project will involve developing a machine learning model using supervised learning to predict future exchange rates for a specific currency pair, based on historical exchange rate data and other relevant features.


In [ ]:
%run pre_processing.py

usdinr_df = pd.read_csv('../data/USDINRX.csv')
forex_df = pd.read_csv('../data/forex.csv')

print('Instantiated the input CSVs as dataframes')

cleaned_usdinr_df = pre_p_usdinr(usdinr_df)
cleaned_forex_df = pre_p_forex(forex_df)

print('Cleaned ' + str(len(forex_df.index) - len(cleaned_forex_df.index)) + ' rows from data/forex.csv')
print('Cleaned ' + str(len(usdinr_df.index) - len(cleaned_usdinr_df.index)) + ' rows from data/USDINRX.csv')

In [ ]:
merged_df = merge(cleaned_forex_df, cleaned_usdinr_df)
print('Merged the two datasets')
merged_df

In [ ]:
merged_x_train, merged_x_test, merged_y_train, merged_y_test = split(merged_df)
print('Merged and did the train/test split of the datasets')

In [6]:
%run lstm_rnn.py
%run rand_network_ensemble.py

lstm_args = { 
    "input_shape": (len(merged_x_train.to_numpy()), len(merged_x_train.to_numpy()[0])), 
    "lstm_units": 32, 
    "dense_units": 16, 
    "output_shape": (1,) 
}
ensembler = RandomNetworkEnsemble(num_networks = 2, base_nn_model = CustomLSTM, model_args = lstm_args, random_seed = 522)

ensembler.train(merged_x_train.to_numpy(), merged_y_train.to_numpy())
predicted_target = ensembler.predict(merged_x_test.iloc[1: , :].to_numpy()) # Have to do .iloc[1: , :] to keep row numbers the same

In [ ]:
#print('RMSE of tuned model\'s predictions: ' + ensembler.calculate_rmse_of_predicted(merged_y_test.iloc[1: , :].to_numpy(), predicted_target))

predicted_target